In [2]:
import requests
from time import sleep
from ecc import PrivateKey
from helper import decode_bech32, decode_base58, hash256, little_endian_to_int
from network import SimpleNode
from script import p2wpkh_script, p2sh_script
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
my_address = private_key.point.bech32_address(testnet=True)
target_amount = 5000000
print(my_address)
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_addresses = '''2N4MoSx2SoExv53GJFEdPuMqL8djPQPH2er'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    if target_address.startswith('2'):
        h160 = decode_base58(target_address)
        script_pubkey = p2sh_script(h160)
    elif target_address.startswith('t'):
        # decode the hash160 from the target address
        _, _, h160 = decode_bech32(target_address)
        # convert hash160 to p2pkh script
        script_pubkey = p2wpkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=script_pubkey,
    ))
change_satoshis = total - fee - output_amount
script_pubkey = p2wpkh_script(private_key.point.hash160())
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, private_key)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))

tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au
01000000000102784364bc58b4fb78e33195ee5bb179d960840654c5bbfaa56169ce1f608a44940100000000ffffffff301e7aa79f6140e8104d721671efee202971ed202539a6925fe79b9c7c6306f00000000000ffffffff02404b4c000000000017a91479e7cf6859a7047b099a078a8ffbbb58b73b863387305b9f02000000001600146e13971913b9aa89659a9f53d327baa8826f2d750248304502210092698bf320b3c29163b1ea211c3e32db25ac52805748adde773afac64d1db37702205a6c7d6f25c3c1c0463f4ad434331ec4a0a35a18d60a0c719ead9609feaa15110121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e02483045022100f03b8651598b1a9cd84cc074d41cfb7c57bb22cec4daeba8cf1ad407dd7a3a900220309041937d6bc65adef2de2970d291f822c1e43415dae1c61764ba8cca2aaff30121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000
success! 9b3e6e253c79672a4e60a8a563ee735c8f87351058a7a0f5d53a2b5771cf6a6d


In [1]:
from ecc import PrivateKey
secret = 8675309
private_key = PrivateKey(secret)
print(private_key.point.p2sh_p2wpkh_address(testnet=True))


2MyJsxLnxj7DsNch4xE7B3nMpB94kDPoE2s
